In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
# 모델 및 토크나이저 로드
model_path = "./model" # 로컬 모델 경로
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

# GPU 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((3072,), eps=1e-05)
    (rotary_emb

In [3]:
# 요약할 텍스트
text = """
뇌-컴퓨터 인터페이스(Brain-Computer Interface, BCI) 기술이 급격히 발전하면서 신경 장애를 가진 환자들에게 새로운 삶의 가능성을 열고 있다.
단순히 운동 기능을 복원하는 수준을 넘어서 일상 생활이나 비디오 게임과 같은 여가 활동을 가능하게 하는 데까지 도달했다.
미국 미시간대 연구진은 심각한 운동 장애를 겪는 환자들이 비디오 게임과 같은 여가 활동을 할 수 있도록 돕는 BCI 기술을 개발했다고 밝혔다.
연구 결과는 국제 학술지 ‘네이처 메디신(Nature Medicine)’에 21일 공개됐다.
BCI 기술은 뇌와 컴퓨터를 연결해 신체 기능을 복원하거나, 인간의 의도를 외부 기기로 전달하는 기술이다.
기존 BCI 기술은 마비 환자의 운동 기능 일부를 복원하는 데 사용돼 왔으나, 손가락 움직임과 같은 정교한 동작을 재현하는 데는 한계가 있었다.
따라서 타이핑이나 악기 연주, 게임 컨트롤러 조작과 같은 활동을 구현하기에는 어려웠다.
연구진은 척수 손상으로 사지마비를 앓는 환자의 뇌에 전극을 이식해 손 움직임과 관련된 신경 신호를 기록했다.
환자가 가상 손의 움직임을 관찰할 때 뇌에서 나오는 신경 데이터를 머신러닝 알고리즘으로 분석해 특정 손가락 움직임과 연관된 신호를 식별했다.
이를 기반으로 뇌에 전극을 이식한 환자가 손가락을 움직이는 것을 상상하는 것만으로 비디오 게임 속 가상 드론을 조종할 수 있게 했다.
환자는 가상 드론의 속도와 방향을 자유롭게 조절하며 여러 장애물을 통과하는 비디오 게임을 성공적으로 수행했다.
"""

# 프롬프트 생성 (요약 지시)
summary_prompt = f"다음 글을 간결하게 요약하세요:\n{text}\n요약:"

# 입력 텍스트 토크나이즈
inputs = tokenizer(summary_prompt, return_tensors="pt", max_length=2048, truncation=True)
inputs = {key: value.to(device) for key, value in inputs.items()}

In [4]:
# 요약 생성
output_ids = model.generate(
    inputs["input_ids"],
    max_new_tokens=150,     # 요약 최대 길이
    temperature=0.7,        # 창의성 조절
    top_p=0.9,              # 확률 분포 컷오프
    repetition_penalty=1.2, # 반복 방지
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [5]:
# 결과 디코딩
summary_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print("요약 결과:", summary_text)

요약 결과: 다음 글을 간결하게 요약하세요:

뇌-컴퓨터 인터페이스(Brain-Computer Interface, BCI) 기술이 급격히 발전하면서 신경 장애를 가진 환자들에게 새로운 삶의 가능성을 열고 있다.
단순히 운동 기능을 복원하는 수준을 넘어서 일상 생활이나 비디오 게임과 같은 여가 활동을 가능하게 하는 데까지 도달했다.
미국 미시간대 연구진은 심각한 운동 장애를 겪는 환자들이 비디오 게임과 같은 여가 활동을 할 수 있도록 돕는 BCI 기술을 개발했다고 밝혔다.
연구 결과는 국제 학술지 ‘네이처 메디신(Nature Medicine)’에 21일 공개됐다.
BCI 기술은 뇌와 컴퓨터를 연결해 신체 기능을 복원하거나, 인간의 의도를 외부 기기로 전달하는 기술이다.
기존 BCI 기술은 마비 환자의 운동 기능 일부를 복원하는 데 사용돼 왔으나, 손가락 움직임과 같은 정교한 동작을 재현하는 데는 한계가 있었다.
따라서 타이핑이나 악기 연주, 게임 컨트롤러 조작과 같은 활동을 구현하기에는 어려웠다.
연구진은 척수 손상으로 사지마비를 앓는 환자의 뇌에 전극을 이식해 손 움직임과 관련된 신경 신호를 기록했다.
환자가 가상 손의 움직임을 관찰할 때 뇌에서 나오는 신경 데이터를 머신러닝 알고리즘으로 분석해 특정 손가락 움직임과 연관된 신호를 식별했다.
이를 기반으로 뇌에 전극을 이식한 환자가 손가락을 움직이는 것을 상상하는 것만으로 비디오 게임 속 가상 드론을 조종할 수 있게 했다.
환자는 가상 드론의 속도와 방향을 자유롭게 조절하며 여러 장애물을 통과하는 비디오 게임을 성공적으로 수행했다.

요약: BCI 기술을 통해 심각한 운동 장애를 가진 환자가 비디오 게임 등을 즐길 수 있는 방법을 찾았다. 환자가 뇌로 전송되는 신경 신호를 분석하고 이를 바탕으로 비ideo 게임에 참여할 수 있도록 하여, 이전에는 불가능했던 활동들을 possible로 만들었다. 이는 future에서는 더 많은 사람들에게 새로운 삶의 가능성을 제공할 것으로 예측된다.


In [6]:
# 2단계: 요약 텍스트로 퀴즈 생성
quiz_prompt = f"다음 요약 내용을 기반으로 주관식 퀴즈와 정답을 생성하세요:\n{summary_text}\n퀴즈:"
quiz_inputs = tokenizer(quiz_prompt, return_tensors="pt", max_length=2048, truncation=True)
quiz_inputs = {key: value.to(device) for key, value in quiz_inputs.items()}

In [ ]:
quiz_ids = model.generate(
    quiz_inputs["input_ids"],
    max_new_tokens=200,
    temperature=0.7,
    top_p=0.9,
    repetition_penalty=1.2,
)

In [ ]:
quiz_output = tokenizer.decode(quiz_ids[0], skip_special_tokens=True)
print("\n생성된 퀴즈 및 정답:\n", quiz_output)